# DigitalTWINS on FHIR - Live demo

## Introduction
The examples considered in this demo involve:
- 2 patients for Exemplar Project 1 – Biomarkers for pulmonary hypertension (note that a simplified workflow with one step/tool is used here).
- 2 patients for Exemplar Project 4 - Breast cancer reporting (6 step workflow).
- 1 patient contributed to both projects (ie in this scenario, they have both pulmonary hypertension and breast cancer).

## Definitions
- FHIR - Fast Healthcare Interoperability Resources
- FHIR Server - a server implemented according to the FHIR standard, allow users to create, update, delete, and search FHIR health data.
- Primary Measurements - a SPARC SDS dataset.
- CWL - Common Workflow Language.
- Workflow tool process - generated by executing a tool/step of a workflow.

## Learning outcomes

- Learn how to install the digitaltwins-on-fhir python client
- Learn how to find all primary measurements for a patient that have been contributed from multiple research studies. This includes:
  -  finding FHIR ImagingStudy resources and their PACS endpoints.
- Learn how to find which workflow and tool generated a specific derived measurement observation.
- Learn how to find all tools and models used by a workflow and their workflow tool processes.
- Learn how to find inputs and outputs of a given tool in a workflow.




## Installing the digitaltwins-on-fhir python client

Install package


*   pip install digitaltwins-on-fhir

In [1]:
pip install digitaltwins-on-fhir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 44.6 MB/s eta 0:00:00


In [2]:
from digitaltwins_on_fhir import Adapter
from pprint import pprint
adapter = Adapter("http://130.216.217.173:8080/fhir")
search = adapter.search()

## Example 1: Finding all primary measurements for a patient

Let's randomly select a patient

Step 1: Find the random patient's uuid and get all dataset composition resources.

In [3]:
measurements = await search.get_patient_measurements("c6923eb4-a5c2-4239-8b7a-16d1268b108d")
pprint(measurements)


[{'datasetName': 'dataset-1',
  'measurements': [{'coding': {'code': '30525-0',
                               'display': None,
                               'system': 'http://loinc.org'},
                    'resourceType': 'Observation',
                    'uuid': 'a1b8afb7-8910-4bf1-80d9-3f84315069d0_c6923eb4-a5c2-4239-8b7a-16d1268b108d_Primary_Measurements_Composition_Observation_0',
                    'value': [{'code': 'a',
                               'system': 'http://unitsofmeasure.org',
                               'unit': 'year',
                               'value': 55.0}]},
                   {'coding': {'code': '8462-4',
                               'display': 'Diastolic blood pressure',
                               'system': 'http://loinc.org'},
                    'resourceType': 'Observation',
                    'uuid': 'a1b8afb7-8910-4bf1-80d9-3f84315069d0_c6923eb4-a5c2-4239-8b7a-16d1268b108d_Primary_Measurements_Composition_Observation_1',
             

# Example 2: Find which workflow, tool, and primary data was used to generate a specific derived measurement observation

It might of interest to find out more information regarding the provenance of a give observation e.g. called "tumour position" (uuid: `231d9946-949a-4fee-8695-5887209bd2db_2673e5a3-8437-41f5-9fef-0983f5662e93_Workflow-Process-Output-Observation-0-0`). For example, we could be interested in finding:
- which assay, workflow and tool generated this observation.

We can start by defining the observation of interest:

In [4]:
res = await search.get_workflow_details_by_derived_data("Observation", "231d9946-949a-4fee-8695-5887209bd2db_2673e5a3-8437-41f5-9fef-0983f5662e93_Workflow-Process-Output-Observation-0-0")
pprint(res)

{'assay': {'name': 'Assay 2: run tumour position selection on duke university '
                   'breast MRI dataset - GUI workflow',
           'uuid': '9ec7a279-a1c8-47b7-9fdd-21b63d95eae0'},
 'patient': 'c6923eb4-a5c2-4239-8b7a-16d1268b108d',
 'process': {'tool': {'name': 'Tumour Position Correction (Manual) Tool',
                      'uuid': '2673e5a3-8437-41f5-9fef-0983f5662e93'},
             'uuid': '231d9946-949a-4fee-8695-5887209bd2db',
             'workflow': {'name': 'Tumour position selection - GUI',
                          'uuid': '4c36c076-3813-4247-8317-e163901b1ae3'}},
 'study': {'name': 'Efficacy assessment of automated tumour position reporting '
                   'workflow (single-site)',
           'uuid': 'f3a36015-4ee1-4596-80d3-4615b74e0904'}}


We can find all inputs and their dataset uuid for generating the Observation

In [5]:
res = await search.get_all_inputs_by_derived_data("Observation", "231d9946-949a-4fee-8695-5887209bd2db_2673e5a3-8437-41f5-9fef-0983f5662e93_Workflow-Process-Output-Observation-0-0")
pprint(res)

{'inputs': [{'datasetUUID': '44680524-3724-407c-84e7-acd92100b33f',
             'description': 'NRRD',
             'inputResourceType': 'ImagingStudy',
             'numberOfSeries': 2,
             'resourceUUID': '4a2455f8-329b-4356-8949-667d3f5f6b5e_9c7f2e1a-5b3d-4f8e-a6c2-1d9b7e4f3a8d_Workflow-Process-Output-ImagingStudy-1-0',
             'url': 'http://130.216.217.175:8042/app/explorer.html#study?uuid=e543e145-2ac8-47c8-a727-9b96c8763b41'},
            {'coding': {'code': '12L:1001',
                        'display': 'tumour position',
                        'system': 'https://www.auckland.ac.nz/en/abi.html'},
             'datasetUUID': 'baac2a4d-beb7-4c40-b557-4d52b3885f14',
             'inputResourceType': 'Observation',
             'resourceUUID': 'baac2a4d-beb7-4c40-b557-4d52b3885f14_c6923eb4-a5c2-4239-8b7a-16d1268b108d_Primary_Measurements_Composition_Observation_0',
             'value': ["{'center': {'x': 119.00000000000001,'y': 193, 'z': "
                       '1

## Example 3: Find all tools and models used by a workflow and their workflow tool processes

Lets find all workflows, and then choose a specific uuid that we can use as an example for finding all tools used by the workflow.

We can also find a workflow by searching by its `name` e.g. "Automated torso model generation - script" direactly.

We can then find all the tools that were used in this workflow including the software and/or models used in the tool (here we show how we can access the workflow from its UUID).

We can also find all the workflow tool process that have been run for that particular workflow.

In [6]:
res = await search.get_all_workflow_tools_by_workflow(name="Automated torso model generation - script")
pprint(res)

{'workflow': {'name': 'Automated torso model generation - script',
              'tools': [{'name': 'create_nifti',
                         'uuid': '69cfadbc-5f52-11ef-917d-484d7e9beb16'},
                        {'name': 'create_nrrd',
                         'uuid': '9c7f2e1a-5b3d-4f8e-a6c2-1d9b7e4f3a8d'},
                        {'name': 'segment',
                         'uuid': '62a0a5dc-39ba-4fe5-81ad-f9eee6b4199e'},
                        {'name': 'create_mesh',
                         'uuid': 'd4f2b8a1-7e6c-49d3-a5f9-8c2e1b7f3d9a'}],
              'uuid': 'e3b3eaa0-65ae-11ef-917d-484d7e9beb16'}}


## Example 4 Find inputs and outputs of a given tool in a workflow
Here we will specify some of the resources that we are looking for, however, all this information can be queried as shown in the previous examples.:

*   workflow uuid: 4c36c076-3813-4247-8317-e163901b1ae3
*   workflow tool uuid: 2673e5a3-8437-41f5-9fef-0983f5662e93

By specifying the workflow and workflow tool UUIDs above, we can retrieve all related assays and workflow tool processes.


Now we have all the processes for that workflow tool.

We can then find the input and output of each workflow tool process:

In [7]:
res = await search.get_all_inputs_outputs_of_workflow_tool(name="Tumour Position Correction (Manual) Tool")
pprint(res)

{'workflow': {'name': 'Tumour position selection - GUI',
              'uuid': '4c36c076-3813-4247-8317-e163901b1ae3'},
 'workflow_tool': {'inputs': [{'data': {'description': 'NRRD',
                                        'inputResourceType': 'ImagingStudy',
                                        'numberOfSeries': 2,
                                        'resourceUUID': '00f762ae-00a4-4f61-9ed3-47e683a9da5a_9c7f2e1a-5b3d-4f8e-a6c2-1d9b7e4f3a8d_Workflow-Process-Output-ImagingStudy-1-0',
                                        'url': 'http://130.216.217.175:8042/app/explorer.html#study?uuid=e4c381f5-0efe-454d-82cb-f01d2f60eb01'},
                               'relevant': {'assay': {'name': 'Assay 2: run '
                                                              'tumour position '
                                                              'selection on '
                                                              'duke university '
                                          